## 무신사 데이터 크롤링

In [1]:
import pandas as pd
import time
import requests

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re

In [95]:
options = Options()
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
options.add_argument("--disable-dev-shm-usage")  # 리소스 제한 방지

driver = webdriver.Chrome(options=options, keep_alive=True)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

In [96]:
driver.get("https://www.musinsa.com/main/musinsa/ranking?storeCode=musinsa&sectionId=200&contentsId=&categoryCode=000&gf=F&period=MONTHLY")
time.sleep(5)


### 테스트
상품 상세페이지는 스크래핑으로 긁어올 수 있는지 확인 -> 불가 -> 셀리니움으로 진행 

```python
href = product_links[0].get_attribute("href")
res = requests.get(href)
soup = BeautifulSoup(res.text, "html.parser")
print(soup.title.text)  # 페이지 제목 출력
print(soup.prettify())

```

In [97]:
def crawl_musinsa_category(driver, top_bar_items, idx, gender="F", max_scrolls=35):
    driver.execute_script("arguments[0].click();", top_bar_items[idx])
    time.sleep(2)

    # 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1.5)

    # 상단으로 이동 버튼 클릭 
    try:
        driver.find_element(By.XPATH, '//button[contains(@class, "_fab__button--white_qazpd_34")]').click()
        time.sleep(1)
    except Exception as e:
        print("상단 이동 실패:", e)
    
    # 테스트용 상품 10개만 가져오기 - 전체 상품 가져오고 싶으면 [:10]
    product_links = driver.find_elements(By.XPATH, '//div[contains(@class, "sc-1t5ihy5-1") and contains(@class, "bLCikV")]/a')[:10]
    hrefs = [link.get_attribute("href") for link in product_links]

    product_data = []
    for href in hrefs:
        driver.execute_script(f"window.open('{href}', '_blank');")
        time.sleep(2)
        driver.switch_to.window(driver.window_handles[-1])

        try:
            product_name = driver.find_element(By.XPATH, './/span[contains(@class, "text-title_18px_med sc-1omefes-1 exqQRL font-pretendard")]').text
        except:
            product_name = ""
        print(product_name)

        try:
            product_brand = driver.find_element(By.XPATH, './/span[contains(@class, "sc-12cqkwk-2 hgzZM")]/span').text
        except:
            product_brand = ""
        print(product_brand)

        try:
            category_spans = driver.find_elements(By.XPATH, './/span[contains(@class, "sc-1prswe3-2 XQBSF")]')
            product_category = " > ".join([span.text.strip() for span in category_spans])
            print(product_category) 
        except:
            product_category = ""
        print(product_category)

        product_rating = None
        product_review_count = None

        try:
            container = driver.find_element(By.XPATH, '//div[contains(@class, "sc-hw7d9p-0 lecuxg gtm-click-button")]')
            
            # 그 안의 span 요소 모두 추출       
            spans = container.find_elements(By.TAG_NAME, 'span')

            for span in spans:
                text = span.text.strip()
                print(text)
                # '후기 1,089개' -> 1089
                if "후기" in text:
                    product_review_count = int(re.sub(r"[^\d]", "", text)) # text에서 숫자(\d)를 제외한 모든 문자([^...])를 빈 문자열로 대체 -> 숫자로 변환
                # '4.8' -> 4.8
                elif re.match(r"^\d+(\.\d+)?$", text): # (\.\d+)? : 소수점과 뒷자리 숫자가 있을 수도 있고 없을 수도 있음
                    product_rating = float(text)
        except Exception as e:
            print("정보 추출 실패:", e)
        
        print(product_rating, product_review_count)

        product_origin_price_elems = driver.find_elements(By.XPATH, './/span[contains(@class, "text-body_13px_med line-through sc-1hw5bl8-3 fyoGBg text-gray-500 font-pretendard")]')
        if product_origin_price_elems:
            product_origin_price = product_origin_price_elems[0].text
        else:
            product_origin_price = None
        print(product_origin_price)

        try:
            product_present_price = driver.find_element(By.XPATH, './/span[contains(@class, "text-title_18px_semi sc-1hw5bl8-7 kXhdZT text-black font-pretendard")]').text
        except:
            product_present_price = ""
        print(product_present_price)

        try:
            product_image_url = driver.find_element(By.XPATH, './/img[contains(@class, "sc-uxvjgl-8 cNQvir")]').get_attribute("src")
        except:
            product_image_url = ""
        print(product_image_url)

        product_data.append({
            "name": product_name,
            "brand": product_brand,
            "category": product_category,
            "rating": product_rating,
            "review_count": product_review_count,
            "origin_price": product_origin_price,
            "present_price": product_present_price,
            "image_url": product_image_url,
            "gender": gender
        })

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    return product_data


In [98]:
product_data = []
top_bar_items = driver.find_elements(By.XPATH, './/button[contains(@class, "sc-ann702-1 jzeDqs gtm-click-button")]')
index = [2] # 카테고리 중 선택할 인덱스 - 신발 선택 (전체 뷰티 신발 상의 아우터 바지 원피스/스커트 가방 패션소품 속옷/홈웨어 스포츠/레저)

for idx in index:
    product_data += crawl_musinsa_category(driver, top_bar_items, idx, gender="F", max_scrolls=40)

df_female = pd.DataFrame(product_data)
df_female.head()

# 모든 창 닫기
driver.quit()

H-Street OG - 블랙:실버 / 403692-02
푸마
신발 > 스니커즈 > 패션스니커즈화
신발 > 스니커즈 > 패션스니커즈화
4.8
후기 130개
4.8 130
None
129,000원
https://image.msscdn.net/thumbnails/images/prd_img/20250507/5091152/detail_5091152_17467535694470_big.png?w=1200
젤-카야노 14 - 화이트:그래파이트 그레이 / 1203A537-110
아식스
신발 > 스니커즈 > 패션스니커즈화
신발 > 스니커즈 > 패션스니커즈화
5
후기 92개
5.0 92
None
179,000원
https://image.msscdn.net/thumbnails/images/prd_img/20250630/5213548/detail_5213548_17512631137505_big.jpg?w=1200
[2팩] FEROA 플리플랍 - 5컬러
23.65
신발 > 샌들/슬리퍼 > 쪼리/플립플랍
신발 > 샌들/슬리퍼 > 쪼리/플립플랍
4.8
후기 269개
4.8 269
118,000원
48,900원
https://image.msscdn.net/thumbnails/images/prd_img/20250509/5097241/detail_5097241_17470168799223_big.jpg?w=1200
[뮬바다 PICK]웨이브 제인 (L/GREY)
디스커버리 익스페디션
신발 > 구두 > 메리제인 슈즈
신발 > 구두 > 메리제인 슈즈
4.9
후기 250개
4.9 250
None
99,000원
https://image.msscdn.net/thumbnails/images/goods_img/20250225/4827479/4827479_17437342462925_big.png?w=1200
Plumpy 헬로키티 참 태그 SET
토앤토
신발 > 샌들/슬리퍼 > 쪼리/플립플랍
신발 > 샌들/슬리퍼 > 쪼리/플립플랍
4.9
후기 324개
4.9 324
None
105,000원
https://ima

In [64]:
df_female.head()

,name,brand,category,rating,review_count,origin_price,present_price,image_url,gender
0,H-Street OG - 블랙:실버 / 403692-02,푸마,신발 > 스니커즈 > 패션스니커즈화,4.8,130,None,"129,000원",https://image.msscdn.net/thumbnails/images/prd...,F
1,젤-카야노 14 - 화이트:그래파이트 그레이 / 1203A537-110,아식스,신발 > 스니커즈 > 패션스니커즈화,5.0,91,None,"179,000원",https://image.msscdn.net/thumbnails/images/prd...,F
2,[2팩] FEROA 플리플랍 - 5컬러,23.65,신발 > 샌들/슬리퍼 > 쪼리/플립플랍,4.8,269,"118,000원","48,900원",https://image.msscdn.net/thumbnails/images/prd...,F
3,[뮬바다 PICK]웨이브 제인 (L/GREY),디스커버리 익스페디션,신발 > 구두 > 메리제인 슈즈,4.9,250,None,"99,000원",https://image.msscdn.net/thumbnails/images/goo...,F
4,Plumpy 헬로키티 참 태그 SET,토앤토,신발 > 샌들/슬리퍼 > 쪼리/플립플랍,4.9,324,None,"105,000원",https://image.msscdn.net/thumbnails/images/prd...,F


In [65]:
df_female.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           10 non-null     object 
 1   brand          10 non-null     object 
 2   category       10 non-null     object 
 3   rating         10 non-null     float64
 4   review_count   10 non-null     int64  
 5   origin_price   2 non-null      object 
 6   present_price  10 non-null     object 
 7   image_url      10 non-null     object 
 8   gender         10 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 852.0+ bytes


In [66]:
df_female.to_csv("musinsa_products_female.csv", index=False, encoding='utf-8-sig')

In [67]:
# 새로운 드라이버로 다시 시작
driver = webdriver.Chrome(options=options, keep_alive=True)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# 남성 페이지로 이동
driver.get("https://www.musinsa.com/main/musinsa/ranking?storeCode=musinsa&sectionId=200&contentsId=&categoryCode=000&period=MONTHLY&gf=M")
time.sleep(5)

# 남성 페이지 크롤링
product_data_male = []

top_bar_items = driver.find_elements(By.XPATH, './/button[contains(@class, "sc-ann702-1 jzeDqs gtm-click-button")]')
index = [2]

for idx in index:
    product_data_male += crawl_musinsa_category(driver, top_bar_items, idx, gender="M", max_scrolls=40)

df_male = pd.DataFrame(product_data_male)
df_male.head()

젤-카야노 14 - 화이트:그래파이트 그레이 / 1203A537-110
아식스
신발 > 스니커즈 > 패션스니커즈화
신발 > 스니커즈 > 패션스니커즈화
5
후기 91개
5.0 91
None
179,000원
https://image.msscdn.net/thumbnails/images/prd_img/20250630/5213548/detail_5213548_17512631137505_big.jpg?w=1200
스웨이드 클로그 HPCV4FA401
슈펜
신발 > 샌들/슬리퍼 > 클로그
신발 > 샌들/슬리퍼 > 클로그
4.7
후기 1,787개
4.7 1787
29,900원
26,910원
https://image.msscdn.net/thumbnails/images/prd_img/20250214/4791040/detail_4791040_17428847606243_big.jpg?w=1200
유트 타우페
이소
신발 > 샌들/슬리퍼 > 스포츠/캐주얼 샌들
신발 > 샌들/슬리퍼 > 스포츠/캐주얼 샌들
4.9
후기 200개
4.9 200
108,000원
102,600원
https://image.msscdn.net/thumbnails/images/goods_img/20250307/4871971/4871971_17465915094522_big.jpg?w=1200
슈퍼스타 II - 블랙:화이트 / JI0079
아디다스
신발 > 스니커즈 > 패션스니커즈화
신발 > 스니커즈 > 패션스니커즈화
5
후기 200개
5.0 200
149,000원
107,290원
https://image.msscdn.net/thumbnails/images/prd_img/20250311/4880137/detail_4880137_17419443579579_big.jpg?w=1200
알파리스폰스 1.0 슬라이드 - 블랙:화이트 / JP5437
아디다스
신발 > 샌들/슬리퍼 > 슬라이드
신발 > 샌들/슬리퍼 > 슬라이드
4.9
후기 243개
4.9 243
59,000원
40,990원
https://image.msscdn.ne

,name,brand,category,rating,review_count,origin_price,present_price,image_url,gender
0,젤-카야노 14 - 화이트:그래파이트 그레이 / 1203A537-110,아식스,신발 > 스니커즈 > 패션스니커즈화,5.0,91,None,"179,000원",https://image.msscdn.net/thumbnails/images/prd...,M
1,스웨이드 클로그 HPCV4FA401,슈펜,신발 > 샌들/슬리퍼 > 클로그,4.7,1787,"29,900원","26,910원",https://image.msscdn.net/thumbnails/images/prd...,M
2,유트 타우페,이소,신발 > 샌들/슬리퍼 > 스포츠/캐주얼 샌들,4.9,200,"108,000원","102,600원",https://image.msscdn.net/thumbnails/images/goo...,M
3,슈퍼스타 II - 블랙:화이트 / JI0079,아디다스,신발 > 스니커즈 > 패션스니커즈화,5.0,200,"149,000원","107,290원",https://image.msscdn.net/thumbnails/images/prd...,M
4,알파리스폰스 1.0 슬라이드 - 블랙:화이트 / JP5437,아디다스,신발 > 샌들/슬리퍼 > 슬라이드,4.9,243,"59,000원","40,990원",https://image.msscdn.net/thumbnails/images/prd...,M


In [68]:
df_male.to_csv("musinsa_products_total.csv", index=False, encoding='utf-8-sig')

In [69]:
# 여성/남성 데이터 합치기
df_total = pd.concat([df_female, df_male], ignore_index=True)
df_total.head()

,name,brand,category,rating,review_count,origin_price,present_price,image_url,gender
0,H-Street OG - 블랙:실버 / 403692-02,푸마,신발 > 스니커즈 > 패션스니커즈화,4.8,130,None,"129,000원",https://image.msscdn.net/thumbnails/images/prd...,F
1,젤-카야노 14 - 화이트:그래파이트 그레이 / 1203A537-110,아식스,신발 > 스니커즈 > 패션스니커즈화,5.0,91,None,"179,000원",https://image.msscdn.net/thumbnails/images/prd...,F
2,[2팩] FEROA 플리플랍 - 5컬러,23.65,신발 > 샌들/슬리퍼 > 쪼리/플립플랍,4.8,269,"118,000원","48,900원",https://image.msscdn.net/thumbnails/images/prd...,F
3,[뮬바다 PICK]웨이브 제인 (L/GREY),디스커버리 익스페디션,신발 > 구두 > 메리제인 슈즈,4.9,250,None,"99,000원",https://image.msscdn.net/thumbnails/images/goo...,F
4,Plumpy 헬로키티 참 태그 SET,토앤토,신발 > 샌들/슬리퍼 > 쪼리/플립플랍,4.9,324,None,"105,000원",https://image.msscdn.net/thumbnails/images/prd...,F


In [70]:
df_total.to_csv("musinsa_products_total.csv", index=False, encoding='utf-8-sig')

In [71]:
df_total.head()

,name,brand,category,rating,review_count,origin_price,present_price,image_url,gender
0,H-Street OG - 블랙:실버 / 403692-02,푸마,신발 > 스니커즈 > 패션스니커즈화,4.8,130,None,"129,000원",https://image.msscdn.net/thumbnails/images/prd...,F
1,젤-카야노 14 - 화이트:그래파이트 그레이 / 1203A537-110,아식스,신발 > 스니커즈 > 패션스니커즈화,5.0,91,None,"179,000원",https://image.msscdn.net/thumbnails/images/prd...,F
2,[2팩] FEROA 플리플랍 - 5컬러,23.65,신발 > 샌들/슬리퍼 > 쪼리/플립플랍,4.8,269,"118,000원","48,900원",https://image.msscdn.net/thumbnails/images/prd...,F
3,[뮬바다 PICK]웨이브 제인 (L/GREY),디스커버리 익스페디션,신발 > 구두 > 메리제인 슈즈,4.9,250,None,"99,000원",https://image.msscdn.net/thumbnails/images/goo...,F
4,Plumpy 헬로키티 참 태그 SET,토앤토,신발 > 샌들/슬리퍼 > 쪼리/플립플랍,4.9,324,None,"105,000원",https://image.msscdn.net/thumbnails/images/prd...,F


In [72]:
df_total.tail()

,name,brand,category,rating,review_count,origin_price,present_price,image_url,gender
15,슈퍼스타 II -블랙:화이트 / JI3538,아디다스,신발 > 스니커즈 > 패션스니커즈화,4.9,70,"159,000원","94,990원",https://image.msscdn.net/thumbnails/images/prd...,M
16,SUB_F161 Black,서브카테고리,신발 > 스니커즈 > 패션스니커즈화,5.0,37,"189,000원","170,100원",https://image.msscdn.net/thumbnails/images/goo...,M
17,XT-6 GTX_L47788400,살로몬,신발 > 스포츠화 > 러닝화,5.0,1,None,"295,000원",https://image.msscdn.net/thumbnails/images/prd...,M
18,H-Street OG - 블랙:실버 / 403692-02,푸마,신발 > 스니커즈 > 패션스니커즈화,4.8,130,None,"129,000원",https://image.msscdn.net/thumbnails/images/prd...,M
19,에바 EVA 남자 여성 쪼리 슬리퍼 샌들 블랙 EVAPT202-BLAC,아일랜드슬리퍼,신발 > 샌들/슬리퍼 > 쪼리/플립플랍,4.8,24,None,"79,000원",https://image.msscdn.net/thumbnails/images/goo...,M


In [73]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           20 non-null     object 
 1   brand          20 non-null     object 
 2   category       20 non-null     object 
 3   rating         20 non-null     float64
 4   review_count   20 non-null     int64  
 5   origin_price   8 non-null      object 
 6   present_price  20 non-null     object 
 7   image_url      20 non-null     object 
 8   gender         20 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.5+ KB
